In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "micls", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9103 | AUPR: 0.6468 | Acc: 95.30%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9139 | AUPR: 0.6582 | Acc: 95.37%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9180 | AUPR: 0.6669 | Acc: 95.45%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9108 | AUPR: 0.6501 | Acc: 95.36%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9152 | AUPR: 0.6592 | Acc: 95.34%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9176 | AUPR: 0.6656 | Acc: 95.40%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9180 | AUPR: 0.6669 | Acc: 95.45%
Internal test: AUC: 0.9275 | AUPR: 0.7058 | Acc: 95.33%
External test: AUC: 0.9152 | AUPR: 0.6584 | Acc: 95.17%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9087 | AUPR: 0.6442 | Acc: 95.29%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9133 | AUPR: 0.6559 | Acc: 95.32%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9174 | AUPR: 0.6665 | Acc: 95.36%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9115 | AUPR: 0.6534 | Acc: 95.30%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9157 | AUPR: 0.6610 | Acc: 95.32%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9192 | AUPR: 0.6685 | Acc: 95.40%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9192 | AUPR: 0.6685 | Acc: 95.40%
Internal test: AUC: 0.9274 | AUPR: 0.7035 | Acc: 95.38%
External test: AUC: 0.9159 | AUPR: 0.6623 | Acc: 95.16%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9113 | AUPR: 0.6442 | Acc: 95.32%
model(C=10.0)
Valid:         AUC: 0.9111 | AUPR: 0.6436 | Acc: 95.30%
model(C=100.0)
Valid:         AUC: 0.9112 | AUPR: 0.6435 | Acc: 95.30%
best model(C=1.0)
Valid:         AUC: 0.9113 | AUPR: 0.6442 | Acc: 95.32%
Internal test: AUC: 0.9209 | AUPR: 0.6878 | Acc: 95.35%
External test: AUC: 0.9075 | AUPR: 0.6233 | Acc: 94.83%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9090 | AUPR: 0.6455 | Acc: 95.41%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9026 | AUPR: 0.6227 | Acc: 94.98%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9024 | AUPR: 0.6232 | Acc: 95.29%
best model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9090 | AUPR: 0.6455 | Acc: 95.41%
Internal test: AUC: 0.9175 | AUPR: 0.6809 | Acc: 95.01%
External test: AUC: 0.9093 | AUPR: 0.6514 | Acc: 95.08%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9097 | AUPR: 0.6543 | Acc: 89.38%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9110 | AUPR: 0.6562 | Acc: 89.31%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9121 | AUPR: 0.6575 | Acc: 89.24%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9101 | AUPR: 0.6563 | Acc: 89.54%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9112 | AUPR: 0.6574 | Acc: 89.48%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9125 | AUPR: 0.6586 | Acc: 89.33%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9101 | AUPR: 0.6563 | Acc: 89.54%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9112 | AUPR: 0.6574 | Acc: 89.48%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9125 | AUPR: 0.6586 | Acc: 89.33%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9125 | AUPR: 0.6586 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9100 | AUPR: 0.6167 | Acc: 94.99%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9104 | AUPR: 0.6112 | Acc: 94.89%
model(kernel=linear, C=1)
Valid:         AUC: 0.9099 | AUPR: 0.6069 | Acc: 94.93%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9104 | AUPR: 0.6112 | Acc: 94.89%
Internal test: AUC: 0.9204 | AUPR: 0.6645 | Acc: 94.87%
External test: AUC: 0.9080 | AUPR: 0.5902 | Acc: 94.56%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9148 | AUPR: 0.6528 | Acc: 95.22%
model(criterion=squared_error)
Valid:         AUC: 0.9147 | AUPR: 0.6543 | Acc: 95.23%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9148 | AUPR: 0.6528 | Acc: 95.22%
Internal test: AUC: 0.9221 | AUPR: 0.6867 | Acc: 95.13%
External test: AUC: 0.9126 | AUPR: 0.6526 | Acc: 95.02%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9109 | AUPR: 0.6325 | Acc: 95.15%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9110 | AUPR: 0.6331 | Acc: 95.15%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9113 | AUPR: 0.6329 | Acc: 95.15%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9113 | AUPR: 0.6329 | Acc: 95.15%
Internal test: AUC: 0.9186 | AUPR: 0.6799 | Acc: 95.06%
External test: AUC: 0.9096 | AUPR: 0.6156 | Acc: 94.68%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7688 | AUPR: 0.4602 | Acc: 95.05%
model(n_neighbors=5)
Valid:         AUC: 0.7874 | AUPR: 0.4946 | Acc: 95.16%
model(n_neighbors=7)
Valid:         AUC: 0.8098 | AUPR: 0.5206 | Acc: 95.28%
best model(n_neighbors=7)
Valid:         AUC: 0.8098 | AUPR: 0.5206 | Acc: 95.28%
Internal test: AUC: 0.8212 | AUPR: 0.5459 | Acc: 95.06%
External test: AUC: 0.8051 | AUPR: 0.5030 | Acc: 94.91%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8660 | AUPR: 0.5715 | Acc: 88.29%
model(criterion=log_loss)
Valid:         AUC: 0.8693 | AUPR: 0.5777 | Acc: 89.13%
model(criterion=entropy)
Valid:         AUC: 0.8693 | AUPR: 0.5777 | Acc: 89.13%
best model(criterion=log_loss)
Valid:         AUC: 0.8693 | AUPR: 0.5777 | Acc: 89.13%
Internal test: AUC: 0.8737 | AUPR: 0.6174 | Acc: 90.85%
External test: AUC: 0.8646 | AUPR: 0.5928 | Acc: 90.30%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8734 | AUPR: 0.4252 | Acc: 86.80%
best model()
Valid:         AUC: 0.8734 | AUPR: 0.4252 | Acc: 86.80%
Internal test: AUC: 0.8792 | AUPR: 0.4507 | Acc: 86.72%
External test: AUC: 0.8686 | AUPR: 0.4006 | Acc: 85.51%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)